# EC analysis - individual image corrections

- this script serves as an addition to the EC control analysis, which allows user to adjust individual parameters of the thresholding funcitons and correct inaccuracies in the individual image masks

In [ ]:
### LIBRARY
import os
import glob
from skimage import io
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from skimage.filters import try_all_threshold
from skimage.filters import threshold_minimum, threshold_isodata, threshold_triangle, threshold_mean
from skimage.filters import threshold_otsu, threshold_li, threshold_yen
from skimage.morphology import binary_dilation
from scipy.ndimage import binary_erosion
from skimage.filters import gaussian
from skimage.morphology import label
from skimage.measure import regionprops
from skimage.morphology import remove_small_objects
from skimage.morphology import remove_small_holes
from statistics import mean, stdev
from natsort import natsorted
from skimage.exposure import equalize_adapthist

In [ ]:
### PARAMETERS
filename = ''

# filename extract
last_folder = os.path.basename(filename)
# output folder:
output_folder = ''

# to save masks:
folder1 = f'{filename}/masks_with_nuclei_count'
folder2 = f'{filename}/masks_nuclei'
os.makedirs(folder1, exist_ok=True)
os.makedirs(folder2, exist_ok=True)

print(last_folder)

# to correctly extract the channels:
nuclei_mask_suffix = '_ch00_mask.tif'
ch405_suffix = '_ch00.tif'
ch594_suffix = '_ch01.tif'
ch647_suffix = '_ch02.tif'

# to correctly calculate the area:
pixel_size_5x = 1.3
pixel_size_10x = 0.65
pixel_size_20x = 0.325
pixel_size_40x = 0.1625
pixel_size_63x = 0.103125

In [ ]:
### LOADING IMAGES

## DAPI in 405 channel:
list_of_dapi_files = natsorted(glob.glob(f"{filename}/*ch00*.tif"))
images_dapi = {}
images_dapi_list = []
for file in list_of_dapi_files:
    img = io.imread(file)
    images_dapi[os.path.basename(file)] = img
    images_dapi_list.append(img)

## 594 channel:
list_of_594_files = natsorted(glob.glob(f"{filename}/*ch01*.tif"))
images_594 = {}
for file in list_of_594_files:
    img = io.imread(file)
    images_594[os.path.basename(file)] = img

## 647 channel:
list_of_647_files = natsorted(glob.glob(f"{filename}/*ch02*.tif"))
images_647 = {}
for file in list_of_647_files:
    img = io.imread(file)
    images_647[os.path.basename(file)] = img

print(f"to check: \n number of 405 images: \t {len(images_dapi)},"
      f"\n number of 594 images: \t {len(images_594)}, \n number of 647 images: \t {len(images_647)}")


In [ ]:
## Grabbing the cellpose masks from the folder

list_of_nuclei_mask_files = natsorted(glob.glob(f"{folder2}/*.tif"))
nuclei_masks = {}

for file in list_of_nuclei_mask_files:
    img = io.imread(file)
    nuclei_masks[os.path.basename(file)] = img

print(f'Number of nuclei masks: {len(nuclei_masks)}')

In [ ]:
### PARAMETERS testing

tested_image = '' #'Image 1'
default_pixel_size = pixel_size_10x

def dapi_analysis(mask):
    thresholding = mask > threshold_triangle(mask)
    dilated = binary_dilation(thresholding, iterations=100) #dilates the individual nuclei by 100 pixels
                                                            #merges the signal together
    thresholding_2 = remove_small_holes(dilated, area_threshold=10000)
    eroded = binary_erosion(thresholding_2, iterations=80) #erodes the expanded mask back to original size
                                                           # ~20px smaller than dilation, because the real  
                                                           #organoid is a bit larger than just nuclei mask
    thresholding_3 = remove_small_objects(eroded, min_size=8000) #removes stray nuclei outside of the organoid
    blurred = gaussian(thresholding_3, sigma=10) #blurrs the jagged edges
    smoothed = blurred > 0.2 #blurring makes the image non-binary, this makes it binary again by selecting
                             #the intensities over certain threshold (in this case everything over 20%)
    return(smoothed)

## 594
def analysis_594(image):
    blurring = gaussian(image,sigma=10) #blurred, because the channel has a lot of debri/unclear signal
                                        #this pools the signal together/removes some of the background
    thresholding = blurring > threshold_li(blurring)
    thresholding_1 = remove_small_objects(thresholding, min_size=5000)
    dilated = binary_dilation(thresholding_1, iterations=60)
    thresholding_2 = remove_small_holes(dilated, area_threshold=100000)
    eroded = binary_erosion(thresholding_2, iterations=60)
    thresholding_3 = remove_small_objects(eroded, min_size=15000)
    return(thresholding_3)

## 647
def analysis_647(image):
    normalisation = equalize_adapthist(image, clip_limit=0.004) #0.009
    thresholding_1 = normalisation > threshold_triangle(normalisation) #NOTE: using a specific threshold 
                                                                       #value is usually fastest for individual images
    thresholding_2 = remove_small_objects(thresholding_1, min_size=150)
    return(thresholding_2)


### Implementation to actual data:

image_dapi_test = nuclei_masks[f'{tested_image}{nuclei_mask_suffix}']
image_thresholded = dapi_analysis(image_dapi_test)

image_594_test = images_594[f'{tested_image}{ch594_suffix}']
thresholded_594_test = analysis_594(image_594_test)

image_647_test = images_647[f'{tested_image}{ch647_suffix}']
thresholded_647_test = analysis_647(image_647_test)


# ## testing of thresholding:
# # tries all automatic thresholding options for the image 
# fig, ax = try_all_threshold(mask_blur_dapi, figsize=(6, 12), verbose=False)
# plt.show()

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12,7))
axs[0,0].imshow(image_dapi_test, cmap='viridis', vmin=0, vmax=15)
axs[0,1].imshow(image_thresholded)
axs[1,0].imshow(thresholded_594_test)
axs[1,1].imshow(thresholded_647_test)

fig, ax = plt.subplots(figsize=(8, 8))
# Show background image
ax.imshow(image_thresholded, cmap='gray')
# Nuclei
ax.imshow(np.ma.masked_where(image_dapi_test == 0, image_dapi_test), cmap='Spectral', vmin=0, vmax=1, alpha=0.8)
# myocardium
ax.imshow(np.ma.masked_where(thresholded_594_test == 0, thresholded_594_test), cmap='magma', alpha=0.4)
# EC
ax.imshow(np.ma.masked_where(thresholded_647_test == 0, thresholded_647_test), cmap="Wistia_r", alpha=1) #vmin=low, vmax=high

In [ ]:
### Analysis

pixel_size = default_pixel_size
print(pixel_size)

### Creation of masks for the individual channels in the image:   
                                                
## 405 (DAPI)
mask_405 = nuclei_masks[f'{tested_image}{nuclei_mask_suffix}'] #_ch00.tif_mask.tif
thresholded_405 = dapi_analysis(mask_405) # organoid mask by blurring nuclei together
nuclei_objects = regionprops(mask_405, img)
## 594 (cTnT)
img_594 = images_594[f'{tested_image}{ch594_suffix}']
myocardium = analysis_594(img_594) & thresholded_405
## 647 (EC)
img_647 = images_647[f'{tested_image}{ch647_suffix}']
thresholded_647 = analysis_647(img_647) & thresholded_405


### Calculations:

#DAPI signal/organoid size
organoid_size = np.sum(thresholded_405) #sum of all pixels, area in pixels^2
organoid_size_in_um = organoid_size*(pixel_size**2) #area in um^2
number_of_nuclei = mask_405.max() #number of nuclei detected by cellpose

# 594 signal/cTnT area
myocardium_area = np.sum(myocardium)
myocardium_area_in_um = myocardium_area*(pixel_size**2)
cTnT_per = myocardium_area/organoid_size

epicardium = thresholded_405 & (~myocardium)
epicardium_size = np.sum(epicardium)
epicardium_size_in_um = epicardium_size*(pixel_size**2)
epicardium_area_per = epicardium_size/organoid_size

#number of nuclei in myocardium:
nuclei_count_myo = 0
nuclei_count_epi = 0

nuclei_sizes_in_um = []

for obj in nuclei_objects:
    coordinates = obj.coords
    nucleus_area = obj.area
    nuclei_sizes_in_um.append(nucleus_area*(pixel_size**2))

    # nuclei in myocardium
    overlap_myo = np.sum(myocardium[coordinates[:,0], coordinates[:,1]])
    overlap_myo_per = overlap_myo/nucleus_area
    # nuclei in epicardium
    overlap_epi = np.sum(epicardium[coordinates[:,0], coordinates[:,1]])
    overlap_epi_per = overlap_epi/nucleus_area

    if overlap_myo_per > 0.5:
        nuclei_count_myo += 1

    if overlap_epi_per > 0.5:
        nuclei_count_epi += 1


if nuclei_count_epi ==0:
    print(f"There is no epicardium in: {tested_image}")

if (nuclei_count_epi+nuclei_count_myo) != number_of_nuclei:
    print(f"{number_of_nuclei-(nuclei_count_epi+nuclei_count_myo)} nuclei are outside of measurement area in: {name_without_last_part}")


# 647 signal/EC area
overal_647_signal = np.sum(thresholded_647)
EC_signal_in_um = overal_647_signal*(pixel_size**2)
EC_area_per = overal_647_signal/organoid_size
EC_area_per_nuclei = EC_signal_in_um/number_of_nuclei


## are ECs in epicardium or myocardium?

# area of signal per number of nuclei in epi/myo
EC_in_epi = thresholded_647 & epicardium #this is an array
EC_in_epi_um = np.sum(EC_in_epi)*(pixel_size**2) #this is pixel area converted to um^2
# np.divide to prevent division by 0
EC_in_epi_per_nuclei = float(np.divide(EC_in_epi_um, nuclei_count_epi, out=np.zeros_like(EC_in_epi_um), where=nuclei_count_epi!=0))
EC_per_in_epicardium = np.sum(EC_in_epi)/overal_647_signal #percentage of ECs in epicardium

EC_in_myo = thresholded_647 & myocardium
EC_in_myo_um = np.sum(EC_in_myo)*(pixel_size**2)
EC_in_myo_per_nuclei = float(np.divide(EC_in_myo_um, nuclei_count_myo, out=np.zeros_like(EC_in_myo_um), where=nuclei_count_myo!=0))
EC_per_in_myocardium = np.sum(EC_in_myo)/overal_647_signal

#miscelaneous:
per_of_myo_nuclei = nuclei_count_myo/number_of_nuclei
per_of_epi_nuclei = nuclei_count_epi/number_of_nuclei

#individual EC data
labelled_647 = label(thresholded_647)
objects_647 = regionprops(labelled_647,img_647)
num_EC_cells = labelled_647.max()

original_areas = []
sphericity = []

for obj in objects_647:
    original_area = obj.area*(pixel_size**2)
    original_areas.append(original_area)
    sphericity.append(obj.eccentricity)

# image mask:
fig, ax = plt.subplots(figsize=(8, 8))

# Show background image
ax.imshow(thresholded_405, cmap='gray')
# Overlay cTnT in magenta
ax.imshow(np.ma.masked_where(myocardium == 0, myocardium), cmap='magma', alpha=0.4)
# Overlay ECs in red
ax.imshow(np.ma.masked_where(thresholded_647 == 0, thresholded_647), cmap='autumn', alpha=0.8)
# Nuclei
ax.imshow(np.ma.masked_where(mask_405 == 0, mask_405), cmap='Blues', vmin=0, vmax=1, alpha=0.8)

ax.set_title(f"Channel masks for image: {tested_image}")
plt.axis('off')
plt.tight_layout()
plt.savefig(f"{folder1}/{tested_image}_corrected_version.png", dpi=300, bbox_inches='tight')
plt.close(fig)


# image data
Individual_image_data = {
    'name': tested_image,
    'organoid area [px^2]': organoid_size,
    'organoid area [um^2]': organoid_size_in_um,
    'number of nuclei': number_of_nuclei,
    'average nuclei size [um^2]': mean(nuclei_sizes_in_um),
    'nuclei sizes stdev': stdev(nuclei_sizes_in_um),
    'myocardium area [px^2]': myocardium_area,
    'myocardium area [um^2]': myocardium_area_in_um,
    'myocardium area [%]': cTnT_per,
    'myocardium nuclei': nuclei_count_myo,
    'myocardium nuclei [%]': per_of_myo_nuclei,
    'epicardium area [px^2]': epicardium_size,
    'epicardium area [um^2]': epicardium_size_in_um,
    'epicardium area [%]': epicardium_area_per,
    'epicardium nuceli': nuclei_count_epi,
    'epicardium nuclei [%]': per_of_epi_nuclei,
    'EC area [um^2]': EC_signal_in_um,
    'EC number': num_EC_cells,
    'EC in epicardium [%]': EC_per_in_epicardium,
    'EC in myocardium [%]': EC_per_in_myocardium,
    'EC per nucleus count': EC_area_per_nuclei,
    'EC in epi per nuclei': EC_in_epi_per_nuclei,
    'EC in myo per nuclei': EC_in_myo_per_nuclei

}

Individual_EC_Data = {
    'EC area': original_areas,
    'EC sphericity': sphericity
}

### DATAFRAME
df_stain1 = pd.DataFrame.from_dict(Individual_image_data, orient='index')
df_stain1 = df_stain1.T
output_path = f'{output_folder}/{last_folder}_{tested_image}.xlsx'
df_stain1.to_excel(output_path)

### DATAFRAME
df_stain2 = pd.DataFrame.from_dict(Individual_image_data, orient='index')
df_stain2 = df_stain2.T
output_path2 = f'{output_folder}/{last_folder}_{tested_image}_individual_ECs.xlsx'
df_stain2.to_excel(output_path2)